In [2]:
from graph_tool.all import *


(ipykernel_launcher.py:78315): Gtk-WARNING **: 14:10:06.678: Locale not supported by C library.
	Using the fallback 'C' locale.
objc[78315]: Class GNotificationCenterDelegate is implemented in both /Users/xihan/miniconda3/envs/gt/lib/libgio-2.0.0.dylib (0x1a3ed6b50) and /usr/local/opt/glib/lib/libgio-2.0.0.dylib (0x1a8340330). One of the two will be used. Which one is undefined.


In [34]:
import os
working_dir = '.' #placeholder

In [27]:
g = Graph(directed=False)
v1 = g.add_vertex()
v2 = g.add_vertex()
v3 = g.add_vertex()
e1 = g.add_edge(v1, v2)
e2 = g.add_edge(v1, v3)

In [28]:
graph_draw(g, vertex_text=g.vertex_index, output="two-nodes.pdf")

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x1a78aa400, at 0x1a75b9fa0>

In [29]:
print(v1.out_degree())
print(e1.source(), e1.target())

2
0 1


In [31]:
vlist = g.add_vertex(10)
print(len(list(vlist)))

10


In [33]:
e3 = g.add_edge(4, 5)

In [37]:
for v in g.iter_vertices():
    print(v)
for e in g.iter_edges():
    print(e)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
[0, 2]
[4, 5]


In [72]:
import numpy as np
import scipy
from scipy.stats import norm
import math

In [101]:
'''
Global configurations
'''
k_alpha = 1
k_beta = .5
E_profit = 10
K_self_coupling = -2

num_clusters = 5
num_agents = 20

In [112]:
from random import randint

'''
Create {num_clusters} random clusters
if cluster_specific = True, use initialise_driving_force() to initialise cluster-specific driving force

Returns: 
           cluster_elements:  {"cluster_i": numpy list of agent ids in cluster i}
           driving_forces:    numpy list of driving forces (properly-/0-initialised)
           agents:            numpy list of agent intrinsics

'''

def create_clusters(num_clusters, num_agents, cluster_specific=False):
    assert num_agents >= num_clusters >= 1
    
    #contains the slices between clusters
    # TODO: needs better random partition
    clusters = []
    for idx in range(num_clusters-1):
        clusters.append(randint(1,num_agents-sum(clusters)-num_clusters+idx))
    clusters.append(num_agents-1)

    # create cluster elements
    agents_ids = np.arange(num_agents)
    np.random.shuffle(agents_ids)
    cluster_elements = {}
    curr_pos = 0
    for i in range(num_clusters):
        cluster_elements["cluster_{}".format(i)] = agents_ids[curr_pos:(clusters[i]+curr_pos-1)]
        curr_pos += clusters[i]
        
    #initialise driving forces
    if cluster_specific:
        driving_forces = initialise_driving_forces(num_agents)
    else:
        driving_forces = np.zeros(num_agents)
        
    #initialise agents intrinsics
    agents = initialise_agents(num_agents)
        
    return cluster_elements, driving_forces, agents
        
    
'''
Initialises the driving forces for each agent in {agents}
You can define initialization schemes here

Retuns: a numpy array containing the driving force of each agent

'''   
def initialise_driving_forces(num_agents):
    driving_forces = np.random.rand(num_agents) #hard-coded for now
    return driving_forces
    
'''
Initialises the intrinsics for each agent in {agents}
You can define initialization schemes here

Retuns: a numpy array containing the intrinsics of each agent
'''
def initialise_agents(num_agents):
    return np.zeros(num_agents) #hard-coded for now



In [113]:
clusters, driving_forces, agents = create_clusters(num_clusters, num_agents)
print("clusters: {}\n\n driving forces: {}\n\n agents:{}\n".format(clusters, driving_forces, agents))

clusters: {'cluster_0': array([ 4, 16,  5, 12,  6,  0, 19, 14, 10,  3,  8, 15, 18,  1]), 'cluster_1': array([], dtype=int64), 'cluster_2': array([], dtype=int64), 'cluster_3': array([], dtype=int64), 'cluster_4': array([13, 17])}

 driving forces: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

 agents:[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]



In [120]:
'''
    updates the agent intrinsics and driving forces 
'''

# TODO: need to fix this
def update_driving_forces(driving_forces, agents, clusters):
    # update each cluster locally
    updated_agents = np.zeros(len(driving_forces))
    for n in range(len(clusters)):
        agents_ids = clusters["cluster_{}".format(n)]
        cluster_size = len(agents_ids)
        
        # update the driving force of each agent in the current cluster
        # based on the other agents in the cluster
        
#         for a in agents_ids:
#             updated_agents[a] = np.sum([x for x in driving_forces[agents_ids] if x != a])
            
#         for a in agents_ids:
#             driving_forces[a] = E_profit + np.sum(updated_agents[agents_ids]) * k_alpha
#             print("agent {} obatined a df of {}".format(a, driving_forces[a]))
#         print("resulting df: {}\n".format(driving_forces))
            
    return updated_agents, driving_forces

In [121]:
upd_df = update_driving_forces(driving_forces, agents, clusters)
upd_df

agent 4 obatined a df of 8716782490.0
agent 16 obatined a df of 8716782490.0
agent 5 obatined a df of 8716782490.0
agent 12 obatined a df of 8716782490.0
agent 6 obatined a df of 8716782490.0
agent 0 obatined a df of 8716782490.0
agent 19 obatined a df of 8716782490.0
agent 14 obatined a df of 8716782490.0
agent 10 obatined a df of 8716782490.0
agent 3 obatined a df of 8716782490.0
agent 8 obatined a df of 8716782490.0
agent 15 obatined a df of 8716782490.0
agent 18 obatined a df of 8716782490.0
agent 1 obatined a df of 8716782490.0
resulting df: [8.71678249e+09 8.71678249e+09 0.00000000e+00 8.71678249e+09
 8.71678249e+09 8.71678249e+09 8.71678249e+09 0.00000000e+00
 8.71678249e+09 0.00000000e+00 8.71678249e+09 0.00000000e+00
 8.71678249e+09 7.00000000e+01 8.71678249e+09 8.71678249e+09
 8.71678249e+09 7.00000000e+01 8.71678249e+09 8.71678249e+09]

resulting df: [8.71678249e+09 8.71678249e+09 0.00000000e+00 8.71678249e+09
 8.71678249e+09 8.71678249e+09 8.71678249e+09 0.00000000e+00
 8.7

(array([6.2262732e+08, 6.2262732e+08, 0.0000000e+00, 6.2262732e+08,
        6.2262732e+08, 6.2262732e+08, 6.2262732e+08, 0.0000000e+00,
        6.2262732e+08, 0.0000000e+00, 6.2262732e+08, 0.0000000e+00,
        6.2262732e+08, 1.4000000e+02, 6.2262732e+08, 6.2262732e+08,
        6.2262732e+08, 1.4000000e+02, 6.2262732e+08, 6.2262732e+08]),
 array([8.71678249e+09, 8.71678249e+09, 0.00000000e+00, 8.71678249e+09,
        8.71678249e+09, 8.71678249e+09, 8.71678249e+09, 0.00000000e+00,
        8.71678249e+09, 0.00000000e+00, 8.71678249e+09, 0.00000000e+00,
        8.71678249e+09, 2.90000000e+02, 8.71678249e+09, 8.71678249e+09,
        8.71678249e+09, 2.90000000e+02, 8.71678249e+09, 8.71678249e+09]))

In [122]:
hi = k_alpha*len(updated_agents) 
lo = None

def normalise_driving_force(driving_forces):
    driving_force_normed = driving_forces/hi
    probability = math.erf(driving_force_normed[0])
    print(driving_force_norm[0], probability)


'''
compute the probability of change given a driving force    
'''
def change_probs(driving_force, hi, lo=None):
    probs = np.zeros(len(driving_force))
    for i in range(len(driving_force)):
        if agents[i] < hi:
            probs[i] = math.erf(driving_force[i])#norm.ppf(agents[i])
        else:
            probs[i] = math.erf(hi)
    return probs


'''
The new opinion {x} of an agent given the prob. of change {p}
'''
def opinion(x, p):
    res = (np.random.rand(1))[0]
    return 1-x if res < p else x


# curretn_state: (agents, probs)
# TODO: adapt this 
def update_state(current_state, f):
    new_state = []
    agents = current_state[0]
    probs = current_state[1]
    for i in range(len(agents)):
        new_state.append(int(f(agents[i], probs[i])))
    return new_state

In [185]:
probs = change_probs(driving_force_norm, hi, lo)

In [192]:
update_state((updated_agents, probs), opinion)

[1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0]